Importing the libraries

In [ ]:
!pip install scikit-learn

In [50]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


Load the dataset

In [51]:
df = pd.read_csv("Social_Network_Ads.csv")

Data pre-processing

In [52]:
# Manual one-hot encoding
df['Gender'] = df['Gender'].map({'Male': 0, 'Female': 1})

# features used for predicting and the actual target variable
X = df[['Gender', 'Age', 'EstimatedSalary']]
y = df['Purchased']

Training and testing data splitting

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Standardise the dataset - for better convergence in admm

In [54]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Activation for regression - manual

In [55]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

A loss function is defined

This is the function to be used in the ADMM

In [56]:
def logistic_loss(X, y, beta, lambda_):
    m = len(y)
    predictions = sigmoid(X @ beta)
    # the following is just a loss function formula
    loss = - (1 / m) * (y @ np.log(predictions + 1e-9) + (1 - y) @ np.log(1 - predictions + 1e-9)) + (lambda_ / 2) * np.sum(beta ** 2)
    # 1e-9 is added to the prediction to avoid log(0) numerical errors
    return loss

ADMM Algorithm

In [67]:
def admm_logistic_regression(X, y, lambda_=1.0, rho=1.0, max_iter=1000, tol=1e-6):

    # lamda_ --> L2 regularization to add penalty to avoid overfitting
    # rho --> parameter for dual updates
    # tol --> convergence tolerance (the algorithm acutally stops if updates are smaller than this value)

    m, n = X.shape

    # model's weights initialized to 0's
    beta = np.random.randn(n) * 0.01
    z = np.zeros(n)
    u = np.zeros(n)

    # Store acc values for plotting
    accuracy_values = []

    for iteration in range(max_iter):
        
        A = X.T @ (sigmoid(X @ beta) - y) + lambda_ * beta + rho * (beta - z + u)
        beta -= np.linalg.pinv(X.T @ X + lambda_ * np.eye(n) + rho * np.eye(n)) @ A

        
        z_old = z.copy()
        z = beta + u
        u += beta - z

        # Compute the loss
        loss = -np.mean(y * np.log(sigmoid(X @ beta)) + (1 - y) * np.log(1 - sigmoid(X @ beta))) + (lambda_ / 2) * np.sum(beta ** 2)

        # Compute accuracy
        predictions = sigmoid(X @ beta) >= 0.5
        accuracy = np.mean(predictions == y)
        accuracy_values.append(accuracy)

        # convergence checking
        primal_residual = np.linalg.norm(beta - z)
        dual_residual = np.linalg.norm(-rho * (z - z_old))
        if primal_residual < tol and dual_residual < tol:
            print(f'The model converged after {iteration + 1} epochs.')
            break
    else:
        print('The model did not converge within 1000 epochs')

    
    
    return beta, accuracy_values

Model fitting

In [68]:
beta_admm, acc_val = admm_logistic_regression(X_train, y_train)
print('Optimized set of weights:', beta_admm)

The model converged after 196 epochs.
Optimized set of weights: [-0.07362575  1.71114091  1.04046282]


Model evaluation

In [61]:
predictions_admm= (sigmoid(X_test @ beta_admm) >= 0.5).astype(int)
accuracy_admm = np.mean(predictions_admm == y_test)
print(f'ADMM Logistic Regression Accuracy: {accuracy_admm * 100:.2f}%')

ADMM Logistic Regression Accuracy: 91.25%
